# LOOSELY SYMMETRIC NAIVE BAYES

## Imports

In [1]:
import warnings
import random
import numpy as np
import pandas as pd
import scipy.sparse as sp
import os
import math
import nltk
import re

from abc import ABCMeta, abstractmethod
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import label_binarize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils import check_X_y, check_array, deprecated
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils.fixes import logsumexp
from sklearn.utils.multiclass import _check_partial_fit_first_call
from sklearn.utils.validation import check_is_fitted, check_non_negative, column_or_1d
from sklearn.utils.validation import _check_sample_weight
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

## Load dataset

In [177]:
mailsDir = "enron1/"
spamDir = os.path.join(mailsDir, "spam")
hamDir = os.path.join(mailsDir, "ham")
print(spamDir)
print(hamDir)
mails = []
hams = []
spams = []
spaminfo = []

i = 0
hamDirList = os.listdir(hamDir)
for file in hamDirList:
    with open(os.path.join(hamDir, file), "r", encoding="latin-1") as f:
        mail = f.read()
        mails.append(mail)
        hams.append(mail)
        spaminfo.append(0)
        i += 1
        if i == 100:
            break
            
print(i)
i = 0
spamDirList = os.listdir(spamDir)
for file in spamDirList:
    with open(os.path.join(spamDir, file), "r", encoding="latin-1") as f:
        mail = f.read()
        mails.append(mail)
        spams.append(mail)
        spaminfo.append(1)
        i += 1
        if i == 100:
            break

print(i)

## Preprocessing

In [2]:
porter_stemmer = nltk.stem.porter.PorterStemmer()

def tokenize(text, stemmer=porter_stemmer):
    lower_text = text.lower()
    tokens = nltk.wordpunct_tokenize(lower_text)
    stems = [porter_stemmer.stem(token) for token in tokens]
    punct_less = [stem for stem in stems if re.match(
        '^[a-zA-Z]+$', stem
    ) is not None]
    return punct_less

# stopwords = nltk.corpus.stopwords.words("english")
# with open("./stopwords.txt", "w") as outf:
#     outf.write("\n".join(stopwords))

with open("./stopwords.txt", "r") as inf:
    stopwords = inf.read().splitlines()

stop_words = []
for word in stopwords:
    stop_words.append(tokenize(word)[0])
stop_words.append("becau")
stop_words = list(dict.fromkeys(stop_words))  # remove duplicates

# binary=True 
# ngram_range=(2,2)

# max_df slouzi k tomu, aby se nevyskytly 0 v deleni pri pocitani feature_log_prob
# min_df je burstiness knockoff
vec = CountVectorizer(
    encoding="latin-1",
    decode_error="replace",
    strip_accents="unicode",
    analyzer="word",
    binary=False,
    stop_words = stop_words,
    tokenizer = tokenize,
    ngram_range=(1,1),
    max_df=0.99,
    min_df=2
)

## Train/test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(mails, spaminfo, test_size = 0.5)

# Count vectorizer
X_train_count = vec.fit_transform(X_train)
print(X_train_count.shape)

# Count vectorizer
X_test_count = vec.transform(X_test)
print(X_test_count.shape)

(2586, 9693)
(2586, 9693)


## Scikit-learn code

In [3]:
class _BaseNB(ClassifierMixin, BaseEstimator, metaclass=ABCMeta):
    """Abstract base class for naive Bayes estimators"""

    @abstractmethod
    def _joint_log_likelihood(self, X):
        """Compute the unnormalized posterior log probability of X
        I.e. ``log P(c) + log P(x|c)`` for all rows x of X, as an array-like of
        shape [n_classes, n_samples].
        Input is passed to _joint_log_likelihood as-is by predict,
        predict_proba and predict_log_proba.
        """

    def _check_X(self, X):
        """To be overridden in subclasses with the actual checks."""
        # Note that this is not marked @abstractmethod as long as the
        # deprecated public alias sklearn.naive_bayes.BayesNB exists
        # (until 0.24) to preserve backward compat for 3rd party projects
        # with existing derived classes.
        return X

    def predict(self, X):
        """
        Perform classification on an array of test vectors X.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        C : ndarray of shape (n_samples,)
            Predicted target values for X
        """
        check_is_fitted(self)
        X = self._check_X(X)
        jll = self._joint_log_likelihood(X)
        return self.classes_[np.argmax(jll, axis=1)]

    def predict_log_proba(self, X):
        """
        Return log-probability estimates for the test vector X.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        C : array-like of shape (n_samples, n_classes)
            Returns the log-probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute :term:`classes_`.
        """
        check_is_fitted(self)
        X = self._check_X(X)
        jll = self._joint_log_likelihood(X)
        # normalize by P(x) = P(f_1, ..., f_n)
        log_prob_x = logsumexp(jll, axis=1)
        return jll - np.atleast_2d(log_prob_x).T

    def predict_proba(self, X):
        """
        Return probability estimates for the test vector X.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        C : array-like of shape (n_samples, n_classes)
            Returns the probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute :term:`classes_`.
        """
        return np.exp(self.predict_log_proba(X))

In [4]:
_ALPHA_MIN = 1e-10

class _BaseDiscreteNB(_BaseNB):
    """Abstract base class for naive Bayes on discrete/categorical data
    Any estimator based on this class should provide:
    __init__
    _joint_log_likelihood(X) as per _BaseNB
    """

    def _check_X(self, X):
        return check_array(X, accept_sparse='csr')

    def _check_X_y(self, X, y):
        return check_X_y(X, y, accept_sparse='csr')

    def _update_class_log_prior(self, class_prior=None):
        n_classes = len(self.classes_)
        if class_prior is not None:
            if len(class_prior) != n_classes:
                raise ValueError("Number of priors must match number of"
                                 " classes.")
            self.class_log_prior_ = np.log(class_prior)
  
        elif self.fit_prior:
            with warnings.catch_warnings():
                # silence the warning when count is 0 because class was not yet
                # observed
                warnings.simplefilter("ignore", RuntimeWarning)
                log_class_count = np.log(self.class_count_)

            # empirical prior, with sample_weight taken into account
            self.class_log_prior_ = (log_class_count -
                                     np.log(self.class_count_.sum()))
            
        else:
            self.class_log_prior_ = np.full(n_classes, -np.log(n_classes))

    def _check_alpha(self):
        if np.min(self.alpha) < 0:
            raise ValueError('Smoothing parameter alpha = %.1e. '
                             'alpha should be > 0.' % np.min(self.alpha))
        if isinstance(self.alpha, np.ndarray):
            if not self.alpha.shape[0] == self.n_features_:
                raise ValueError("alpha should be a scalar or a numpy array "
                                 "with shape [n_features]")
        if np.min(self.alpha) < _ALPHA_MIN:
            warnings.warn('alpha too small will result in numeric errors, '
                          'setting alpha = %.1e' % _ALPHA_MIN)
            return np.maximum(self.alpha, _ALPHA_MIN)
        return self.alpha

    def fit(self, X, y, sample_weight=None):

        self.X = X
        self.y = y
        X, y = self._check_X_y(X, y)
        _, n_features = X.shape
        self.n_features_ = n_features

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # We convert it to np.float64 to support sample_weight consistently;
        # this means we also don't have to cast X to floating point
        if sample_weight is not None:
            Y = Y.astype(np.float64, copy=False)
            sample_weight = np.asarray(sample_weight)
            sample_weight = np.atleast_2d(sample_weight)
            Y *= check_array(sample_weight).T

        class_prior = self.class_prior

        # Count raw events from data before updating the class log prior
        # and feature log probas
        n_effective_classes = Y.shape[1]

        self._init_counters(n_effective_classes, n_features)
        # added y here as a parameter, to be used in LSNB implementation of _count()
        self._count(X, Y, y)
        alpha = self._check_alpha()
        self._update_feature_log_prob(alpha)
        self._update_class_log_prior(class_prior=class_prior)
        return self

    def _init_counters(self, n_effective_classes, n_features):
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.feature_count_ = np.zeros((n_effective_classes, n_features),
                                       dtype=np.float64)

    # XXX The following is a stopgap measure; we need to set the dimensions
    # of class_log_prior_ and feature_log_prob_ correctly.
    def _get_coef(self):
        return (self.feature_log_prob_[1:]
                if len(self.classes_) == 2 else self.feature_log_prob_)

    def _get_intercept(self):
        return (self.class_log_prior_[1:]
                if len(self.classes_) == 2 else self.class_log_prior_)

    coef_ = property(_get_coef)
    intercept_ = property(_get_intercept)

    def _more_tags(self):
        return {'poor_score': True}

In [5]:
class LooselySymmetricNB(_BaseDiscreteNB):
    
    def __init__(self, alpha=1.0, fit_prior=True, class_prior=None, enhance=False):
        self.alpha = alpha
        self.fit_prior = fit_prior
        self.class_prior = class_prior
        self.enhance = enhance
        
    def _count(self, X, Y, y):
        """Count and smooth feature occurrences."""
        
        check_non_negative(X, "LooselySymmetricNB (input X)")
        self.feature_count_ += safe_sparse_dot(Y.T, X)
        self.class_count_ += Y.sum(axis=0)
        
        # we need these two values in order to calculate document frequency in _calculate_df()
        self.X = X
        self.y = y
        
    def _update_feature_log_prob(self, alpha):
        ### VLIV PARAMETRU ALPHA?
        self.smoothed_fc = self.feature_count_ + alpha
        self.smoothed_cc = self.smoothed_fc.sum(axis=1)
        # funguje i radek nize?
#         self.smoothed_cc = self.feature_count_.sum(axis=1)
        
        self._calculate_df()
        self._calculate_abcd(self.smoothed_fc, self.smoothed_cc.reshape(-1, 1), self.enhance)
        
        # HAM
        self.bd = (self.b * self.d) / (self.b + self.d)
        self.ac = (self.a * self.c) / (self.a + self.c)
        bd = (self.b * self.d) / (self.b + self.d)
        ac = (self.a * self.c) / (self.a + self.c)
        numerator = self.a + bd
        denumerator = self.a + self.b + ac + bd
        
        # 2d pole, index 0 je pole ham, index 1 je pole spam
        self.feature_log_prob_ = np.empty(self.feature_count_.shape) 
#         self.feature_log_prob_[0] = numerator / denumerator
        self.feature_log_prob_[0] = np.log(numerator) - np.log(denumerator)
        
        # SPAM
        numerator = self.c + bd
        denumerator = self.c + self.d + ac + bd
        
#         self.feature_log_prob_[1] = numerator / denumerator
        self.feature_log_prob_[1] = np.log(numerator) - np.log(denumerator)
    
    def _calculate_df(self):
        
        self.df = np.zeros(self.feature_count_.shape, dtype=np.int32)
        for mail_idx, mail in enumerate(self.X.toarray()):
            for word_idx, word in enumerate(mail):
                if word >= 1:
                    self.df[self.y[mail_idx]][word_idx] += 1
    
    def _calculate_abcd(self, fc, cc, enhance):
        
        # at 0 is ham info, at 1 is spam info
        if enhance:
            word_density_ham = fc[0] / cc[0]
            word_density_spam = fc[1] / cc[1]
        
        else:
            word_density_ham = 1
            word_density_spam = 1
        
        self.a = (self.df[0] / self.class_count_[0]) * word_density_ham
        self.b = (1 - self.a) * word_density_spam
        self.c = (self.df[1] / self.class_count_[1]) * word_density_spam
        self.d = (1 - self.c) * word_density_ham
        
        
    def _joint_log_likelihood(self, X):
       
        return (safe_sparse_dot(X, self.feature_log_prob_.T) + 
                self.class_log_prior_)

## Validation

In [65]:
mailsDir = "enron6/"
spamDir = os.path.join(mailsDir, "spam")
hamDir = os.path.join(mailsDir, "ham")
print(spamDir)
print(hamDir)
mails = []
# hams = []
# spams = []
spaminfo = []

hamDirList = os.listdir(hamDir)
for file in hamDirList:
    with open(os.path.join(hamDir, file), "r", encoding="latin-1") as f:
        mail = f.read()
        mails.append(mail)
#         hams.append(mail)
        spaminfo.append(0)

spamDirList = os.listdir(spamDir)
for file in spamDirList:
    with open(os.path.join(spamDir, file), "r", encoding="latin-1") as f:
        mail = f.read()
        mails.append(mail)
#         spams.append(mail)
        spaminfo.append(1)

ordered = list(zip(mails, spaminfo))
random.shuffle(ordered)
mails, spaminfo = zip(*ordered)
print("loaded")

enron6/spam
enron6/ham
loaded


### BernoulliNB

In [68]:
from sklearn.naive_bayes import BernoulliNB
vec.binary = True

for size in [50, 100, 200, 300, 400, 500, 600]:
    X_train, X_test, y_train, y_test = train_test_split(mails[:size], spaminfo[:size], test_size = 0.5)
    
    # Count vectorizer
    X_train_count = vec.fit_transform(X_train)
    print(f"{X_train_count.shape[0] *2}")
    spam_size_train = sum(y_train) / len(y_train)
#     print(f"spam size train: {spam_size_train}")

    # Count vectorizer
    X_test_count = vec.transform(X_test)
#     print(f"X_test_count.shape: {X_test_count.shape}")
    spam_size_test = sum(y_test) / len(y_test)
#     print(f"spam size test: {spam_size_test}")

    bnb = BernoulliNB(class_prior=[0.5, 0.5])
    bnb.fit(X_train_count, y_train)

    print(f"{bnb.score(X_test_count.toarray(), y_test)}")
    print(f"{precision_score(y_test, bnb.predict(X_test_count.toarray()))}")
    print(f"{recall_score(y_test, bnb.predict(X_test_count.toarray()))}")
    print(f"{f1_score(y_test, bnb.predict(X_test_count.toarray()))}")
    print("-----------")

50
0.84
0.875
0.9545454545454546
0.9130434782608695
-----------
100
0.76
0.8043478260869565
0.925
0.8604651162790697
-----------
200
0.77
0.8152173913043478
0.9259259259259259
0.8670520231213872
-----------
300
0.8066666666666666
0.8169014084507042
0.9747899159663865
0.888888888888889
-----------
400
0.905
0.8983050847457628
0.99375
0.9436201780415431
-----------
500
0.82
0.8185840707964602
0.9788359788359788
0.8915662650602411
-----------
600
0.84
0.8484848484848485
0.9655172413793104
0.9032258064516129
-----------


### GaussianNB, MultinomialNB, LSNB, eLSNB

In [41]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
vec.binary = False

for size in [50, 100, 200, 300, 400, 500, 600]:
    X_train, X_test, y_train, y_test = train_test_split(mails[:size], spaminfo[:size], test_size = 0.5)
    
    # Count vectorizer
    X_train_count = vec.fit_transform(X_train)
#     print(f"X_train_count.shape: {X_train_count.shape[0]}")
#     print(f"{X_train_count.shape[0] * 2}")
    spam_size_train = sum(y_train) / len(y_train)
#     print(f"spam size train: {spam_size_train}")

    # Count vectorizer
    X_test_count = vec.transform(X_test)
#     print(f"X_test_count.shape: {X_test_count.shape}")
    spam_size_test = sum(y_test) / len(y_test)
#     print(f"spam size test: {spam_size_test}")
    
    gnb = GaussianNB(priors=[0.5, 0.5])
    gnb.fit(X_train_count.toarray(), y_train)
    mnb = MultinomialNB(class_prior=[0.5, 0.5])
    mnb.fit(X_train_count, y_train)
    lsnb = LooselySymmetricNB(class_prior=[0.5, 0.5])
    lsnb.fit(X_train_count, y_train)
    elsnb = LooselySymmetricNB(class_prior=[0.5, 0.5], enhance=True)
    elsnb.fit(X_train_count, y_train)

    
    print("LSNB")
    print(f"{X_train_count.shape[0] * 2}")
    print(f"{lsnb.score(X_test_count.toarray(), y_test)}")
    print(f"{precision_score(y_test, lsnb.predict(X_test_count.toarray()))}")
    print(f"{recall_score(y_test, lsnb.predict(X_test_count.toarray()))}")
    print(f"{f1_score(y_test, lsnb.predict(X_test_count.toarray()))}")
    print("-----------")
    
    print("ENHA")
    print(f"{X_train_count.shape[0] * 2}")
    print(f"{elsnb.score(X_test_count.toarray(), y_test)}")
    print(f"{precision_score(y_test, elsnb.predict(X_test_count.toarray()))}")
    print(f"{recall_score(y_test, elsnb.predict(X_test_count.toarray()))}")
    print(f"{f1_score(y_test, elsnb.predict(X_test_count.toarray()))}")
    print("-----------")
    
    print("GAUSSIAN")
    print(f"{X_train_count.shape[0] * 2}")
    print(f"{gnb.score(X_test_count.toarray(), y_test)}")
    print(f"{precision_score(y_test, gnb.predict(X_test_count.toarray()))}")
    print(f"{recall_score(y_test, gnb.predict(X_test_count.toarray()))}")
    print(f"{f1_score(y_test, gnb.predict(X_test_count.toarray()))}")
    print("-----------")

    print("MNB")
    print(f"{X_train_count.shape[0] * 2}")
    print(f"{mnb.score(X_test_count.toarray(), y_test)}")
    print(f"{precision_score(y_test, mnb.predict(X_test_count.toarray()))}")
    print(f"{recall_score(y_test, mnb.predict(X_test_count.toarray()))}")
    print(f"{f1_score(y_test, mnb.predict(X_test_count.toarray()))}")
    print("==============")
    print("==============")

LSNB
50
0.88
0.8
0.8888888888888888
0.8421052631578948
-----------
ENHA
50
0.88
0.8
0.8888888888888888
0.8421052631578948
-----------
GAUSSIAN
50
0.72
0.75
0.3333333333333333
0.46153846153846156
-----------
MNB
50
0.84
0.7272727272727273
0.8888888888888888
0.7999999999999999
LSNB
100
0.9
0.8076923076923077
1.0
0.8936170212765957
-----------
ENHA
100
0.92
0.84
1.0
0.9130434782608696
-----------
GAUSSIAN
100
0.7
0.8
0.38095238095238093
0.5161290322580645
-----------
MNB
100
0.9
0.8636363636363636
0.9047619047619048
0.8837209302325582
LSNB
200
0.83
0.6875
0.9428571428571428
0.7951807228915663
-----------
ENHA
200
0.89
0.8
0.9142857142857143
0.8533333333333333
-----------
GAUSSIAN
200
0.89
0.9
0.7714285714285715
0.8307692307692307
-----------
MNB
200
0.9
0.8205128205128205
0.9142857142857143
0.8648648648648648
LSNB
300
0.8933333333333333
0.7540983606557377
0.9787234042553191
0.8518518518518519
-----------
ENHA
300
0.8866666666666667
0.734375
1.0
0.8468468468468469
-----------
GAUSSIAN
300
